In [3]:
%load_ext autoreload
%autoreload 2
import logging

logging.getLogger('mat73').setLevel(logging.CRITICAL)
import os
import re
from scipy.io import loadmat
import mat73
logging.getLogger('mat73').setLevel(logging.CRITICAL)

import warnings

from collections import Counter
import pandas as pd
import numpy as np
from scipy.stats import f_oneway
from sklearn import linear_model

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

from utils import *
from connectivity_dynamics import *
from eibal import *
from plotpowerbal import get_involved_dict
matplotlib.use('TkAgg')

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
COLOR_MAP

{'pz': '#D95319',
 'soz': '#A2142F',
 'nz': '#0072BD',
 'niz': '#0072BD',
 'pz_soz': '#D95319',
 'soz_soz': '#A2142F',
 'nz_soz': '#0072BD',
 'niz_soz': '#A2142F',
 'niz_pz': '#D95319',
 'niz_niz': '#0072BD',
 'PZ': '#D95319',
 'SOZ': '#A2142F',
 'NZ': '#0072BD',
 'NIZ': '#0072BD',
 'PZ_SOZ': '#D95319',
 'SOZ_SOZ': '#A2142F',
 'NZ_SOZ': '#0072BD',
 'NIZ_SOZ': '#A2142F',
 'NIZ_PZ': '#D95319',
 'NIZ_NIZ': '#0072BD'}

In [117]:
POW_DIR = "/mnt/ernie_main/Ghassan/ephys/data/ei_bal/"
SPES_DIR = "/mnt/ernie_main/Ghassan/ephys/data/spes/"
tst_spes = "Epat26_3mA_SPES_windowed_PSD_300_length_10_stride_DISTGAP9999mm.mat"
tst_spes_struct = load_mat(os.path.join(SPES_DIR, "Epat26", tst_spes))
tst_spes_struct['spes']['pat_spes'].keys()
tst_spes_struct = tst_spes_struct['spes']['pat_spes']
tst_spes_struct.keys()

dict_keys(['currents', 'data_RBP_SUB', 'data_RBP_SUB_AllDists', 'data_RBP_Z', 'data_RBP_Z_AllDists', 'data_SUB', 'data_SUB_AllDists', 'data_Z', 'data_Z_AllDists', 'data_axis_labels_for_each_current', 'dist_threshes_mm', 'mean_stim_pretrain_PSD', 'patID_clean', 'psd_freqs', 'rbp_bands', 'response_labels', 'response_soz', 'std_stim_pretrain_PSD', 'stim_labels', 'stim_soz', 'time_win_ms'])

In [120]:
tst_spes_struct["data_Z_AllDists"].shape

(53, 103, 150)

In [6]:
#Exploring spes sruct
# bands = tst_spes_struct['spes']['pat_spes']['rbp_bands']
# stim_labels = tst_spes_struct['spes']['pat_spes']['stim_labels']
# resp_labels = tst_spes_struct['spes']['pat_spes']['response_labels']
# print(f"Bands of interest {bands}")
# print(f" {len(stim_labels)} Stim labels: { stim_labels}")


# print(f"{len(resp_labels)}Response labels: {tst_spes_struct['spes']['pat_spes']['response_labels']}
#")


In [74]:
stim_labels = [s[0] for s in tst_spes_struct['stim_labels']]
resp_labels = tst_spes_struct['response_labels']
resp_labels = [r[0] for r in resp_labels]
stim_inds = [i for i, e in enumerate(resp_labels) if e in stim_labels]
z_freq = tst_spes_struct['data_RBP_Z_AllDists']
symm_conn = z_freq[:,stim_inds,:]


In [78]:
i = 5

symm_conn[i,:,:] - symm_conn[:,i,:]

array([[ 1.2138835e+00, -3.2175124e-01, -6.0703182e-01, -8.0313718e-01,
        -7.7681148e-01],
       [ 1.3409702e+00,  1.0481823e-01, -2.5085843e-01, -2.3430395e+00,
        -1.4442296e+00],
       [ 7.4646556e-01, -1.1885046e+00, -1.1149440e+00, -5.2255499e-01,
        -7.5879407e-01],
       [-8.1403196e-01, -1.2331476e+00,  9.9256814e-01,  1.5646241e+00,
         1.0503101e+00],
       [ 2.5446546e-01,  8.3731353e-01,  2.5512278e-04, -3.0926052e-01,
        -2.4285316e-03],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00],
       [ 1.7037364e+00,  2.3889661e-01, -1.1664362e+00, -1.8940797e+00,
        -2.0342107e+00],
       [ 2.2793841e-01, -7.2915787e-01, -7.5368422e-01, -2.1823138e-01,
        -3.9778355e-01],
       [-2.5448391e+00, -1.1111491e+00,  1.0190909e+00,  2.8288765e+00,
         3.0953686e+00],
       [ 2.1657690e-01, -6.6566050e-02, -4.5229107e-01,  3.2514393e-02,
        -9.3600428e-01],
       [ 7.9726201e-01, -1.008

True

In [122]:
def stim_to_df(stim_struct, symmetric=False, data_col ='data_RBP_Z_AllDists' ):
    patID = stim_struct['patID_clean']
    stim_labels = [s[0] for s in stim_struct['stim_labels']]
    resp_labels = [r[0] for r in stim_struct['response_labels']]
    z_freq = stim_struct[data_col]



    resp_soz = [map_label(resp) for resp in stim_struct['response_soz']]
    stim_soz = [map_label(stim) for stim in stim_struct['stim_soz']]
    resp_dict = defaultdict(lambda : [])
    if symmetric:
        assert set(stim_labels).issubset(set(resp_labels)), "Some stim labels not in resp labels!"
        stim_inds = [i for i, e in enumerate(resp_labels) if e in stim_labels]
        resp_labels = [resp_labels[i] for i in stim_inds]
        z_freq = z_freq[:,stim_inds,:]
        resp_soz = [resp_soz[i] for i in stim_inds]
    for i, stim_site in enumerate(stim_labels):
        for j, band in enumerate(BANDS[2:]):
            resp_dict["stim_bip"].extend([stim_site]*len(resp_labels))
            resp_dict['resp_bip'].extend(resp_labels)
            conn_mat = z_freq[i,:,j]
            if symmetric:
                net_z = conn_mat - z_freq[:,i,j]
                resp_dict['net_change_z'].extend(net_z)
            resp_dict[f'change_z'].extend(conn_mat)
            resp_dict['band'].extend([band]*len(resp_labels))
            resp_dict['resp_label'].extend(resp_soz)
            resp_dict['stim_label'].extend([stim_soz[i]]*len(resp_labels))
    stim_df = pd.DataFrame.from_dict(resp_dict)
    stim_df['patID'] = patID
    stim_df['stim_resp'] = stim_df.apply(lambda x: f"{x['stim_label']}_{x['resp_label']}", axis=1)
    return stim_df

def get_general_involved(pow_df):
    involved_dict = defaultdict(lambda : True)
    for event in pow_df.eventID.unique():
        pow_event_df = pow_df[pow_df.eventID == event]
        pow_event_dict = get_involved_dict(pow_event_df,band='beta')
        for key in pow_event_dict.keys():
            involved_dict[key] = involved_dict[key] and pow_event_dict[key]
    return involved_dict

def merge_pow_spes(pow_df, spes_df, **kwargs):

    involved_dict = get_general_involved(pow_df)
    spes_df['beta_involved'] = spes_df.stim_bip.apply(lambda x: involved_dict[x])
    return spes_df

def load_merge_spes_subj(spes_fname, pow_fname, **kwargs):
    spes_struct = load_mat(spes_fname)
    stim_df = stim_to_df(spes_struct['spes']['pat_spes'], **kwargs)
    pow_df = pd.read_csv(pow_fname)
    spes_pow_df = merge_pow_spes(pow_df, stim_df, **kwargs)
    return spes_pow_df

def load_merge_spes(subjects,pow_dir, spes_dir, **kwargs):
    spes_dfs =[]
    for subject in subjects:
        try:
            pow_fname = os.path.join(pow_dir, f"power_bal_{subject}_centered.csv")
            assert os.path.exists(pow_fname)
            spes_fname = os.path.join(spes_dir, subject, f"{subject}_3mA_SPES_windowed_PSD_300_length_10_stride_DISTGAP9999mm.mat")
            assert os.path.exists(spes_fname)
        except AssertionError:
            logger.warning(f"Subject {subject} not found in spes and power!")
            logger.warning(f"Pow status: {os.path.exists(pow_fname)} true for {pow_fname}")
            logger.warning(f"SPES status: {os.path.exists(spes_fname)} for {spes_fname}")
            continue
        df = load_merge_spes_subj(spes_fname, pow_fname, **kwargs)
        spes_dfs.append(df)
    return pd.concat(spes_dfs)

In [123]:
spes_subjects = pd.read_csv("../data/spes_subj.csv",header=None)
spes_subjects = spes_subjects[0].values
spes_df = load_merge_spes(spes_subjects, POW_DIR, SPES_DIR, symmetric = True, data_col = 'data_Z_AllDists')

/tmp/ipykernel_147895/1690869941.py:53: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pow_df = pd.read_csv(pow_fname)
/tmp/ipykernel_147895/1690869941.py:53: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pow_df = pd.read_csv(pow_fname)
/tmp/ipykernel_147895/1690869941.py:53: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pow_df = pd.read_csv(pow_fname)


In [104]:
spes_df

,stim_bip,resp_bip,net_change_z,change_z,band,resp_label,stim_label,patID,stim_resp,beta_involved
0,LA1LA2,LTP1LTP2,0.000000,0.481620,alpha,SOZ,NIZ,Epat26,NIZ_SOZ,False
1,LA1LA2,LTP5LTP6,1.546933,1.043761,alpha,NIZ,NIZ,Epat26,NIZ_NIZ,False
2,LA1LA2,LTP7LTP8,-0.002365,0.102830,alpha,NIZ,NIZ,Epat26,NIZ_NIZ,False
3,LA1LA2,LAM1LAM2,-1.062616,0.356209,alpha,SOZ,NIZ,Epat26,NIZ_SOZ,False
4,LA1LA2,LAM3LAM4,-0.207570,-0.242355,alpha,NIZ,NIZ,Epat26,NIZ_NIZ,False
...,...,...,...,...,...,...,...,...,...,...
11659,RTF6RTF7,LAC1LAC2,0.377422,-0.235091,gamma_H,NIZ,NIZ,Spat52,NIZ_NIZ,True
11660,RTF6RTF7,LAC3LAC4,0.339572,0.748802,gamma_H,PZ,NIZ,Spat52,NIZ_PZ,True
11661,RTF6RTF7,LAC5LAC6,0.365234,0.649705,gamma_H,NIZ,NIZ,Spat52,NIZ_NIZ,True
11662,RTF6RTF7,LAC7LAC8,-0.382265,0.082931,gamma_H,NIZ,NIZ,Spat52,NIZ_NIZ,True


In [111]:

cols =['patID','band','stim_resp','beta_involved','change_z', 'net_change_z']
spes_df[cols]

,patID,band,stim_resp,beta_involved,change_z,net_change_z
0,Epat26,alpha,NIZ_SOZ,False,0.481620,0.000000
1,Epat26,alpha,NIZ_NIZ,False,1.043761,1.546933
2,Epat26,alpha,NIZ_NIZ,False,0.102830,-0.002365
3,Epat26,alpha,NIZ_SOZ,False,0.356209,-1.062616
4,Epat26,alpha,NIZ_NIZ,False,-0.242355,-0.207570
...,...,...,...,...,...,...
11659,Spat52,gamma_H,NIZ_NIZ,True,-0.235091,0.377422
11660,Spat52,gamma_H,NIZ_PZ,True,0.748802,0.339572
11661,Spat52,gamma_H,NIZ_NIZ,True,0.649705,0.365234
11662,Spat52,gamma_H,NIZ_NIZ,True,0.082931,-0.382265


In [112]:
spes_summary_df = spes_df[cols].groupby(['patID', 'stim_resp', 'beta_involved', 'band']).mean().reset_index()
spes_summary_df

,patID,stim_resp,beta_involved,band,change_z,net_change_z
0,Epat26,NIZ_NIZ,False,alpha,0.205764,-0.071422
1,Epat26,NIZ_NIZ,False,beta,0.109171,-0.007757
2,Epat26,NIZ_NIZ,False,gamma_H,-0.007211,0.119606
3,Epat26,NIZ_NIZ,False,gamma_l,-0.235235,0.031288
4,Epat26,NIZ_NIZ,True,alpha,0.016474,-0.179073
...,...,...,...,...,...,...
1871,Spat52,SOZ_SOZ,False,gamma_l,-0.496938,-0.552686
1872,Spat52,SOZ_SOZ,True,alpha,-0.363209,-0.072034
1873,Spat52,SOZ_SOZ,True,beta,-0.588959,-0.746108
1874,Spat52,SOZ_SOZ,True,gamma_H,32.699669,32.589409


In [113]:
keep_rel =['NIZ_NIZ', 'NIZ_PZ', 'NIZ_SOZ', 'PZ_NIZ', 'PZ_PZ', 'PZ_SOZ',
       'SOZ_NIZ', 'SOZ_PZ', 'SOZ_SOZ']

In [124]:
plot_df = spes_summary_df[spes_summary_df.stim_resp.isin(keep_rel)]
rename = labels={False:'Low Beta Recruitment', True:"High Beta Recruitment"}
plot_df.beta_involved = plot_df.beta_involved.apply(lambda x : rename[x])
for band in plot_df.band.unique():
    plt.close()
    plt.figure(figsize=(11,5))
    ax= sns.violinplot(data=plot_df[plot_df.band==band], x='stim_resp', y='net_change_z', hue='beta_involved')
    #TODO extend whiskers
    #TODO visualize hypothesis test 
    plt.title(f"SPES change in Relative {band} Power")
    plt.xlabel("Stim-Response Pairs")
    plt.ylabel(f"Change in {band} power")
    plt.legend()
    plt.savefig(f"../viz/spes/SPES_{band}_boxplot.pdf",transparent = True)

## not getting results that make sense with ISH


/tmp/ipykernel_147895/1592995421.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plot_df.beta_involved = plot_df.beta_involved.apply(lambda x : rename[x])


## ISH Recapitulate Results

In [125]:
keep_rel =['NIZ_NIZ', 'NIZ_PZ', 'NIZ_SOZ' ]

plot_df = spes_summary_df[spes_summary_df.stim_resp.isin(keep_rel)]
rename = labels={False:"same", True:"same"}
plot_df.beta_involved = plot_df.beta_involved.apply(lambda x : rename[x])
for band in plot_df.band.unique():
    plt.close()
    plt.figure(figsize=(11,5))
    ax= sns.violinplot(data=plot_df[plot_df.band==band], x='stim_resp', y='net_change_z', hue='stim_resp', palette=COLOR_MAP)
    #TODO extend whiskers
    #TODO visualize hypothesis test 
    plt.title(f"SPES change in Relative {band} Power")
    plt.xlabel("Stim-Response Pairs")
    plt.ylabel(f"Change in {band} power")
    plt.legend()
    plt.savefig(f"../viz/spes/SPES_ISH_{band}_boxplot.pdf",transparent = True)

## not getting results that make sense with ISH



/tmp/ipykernel_147895/3542202393.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plot_df.beta_involved = plot_df.beta_involved.apply(lambda x : rename[x])
/tmp/ipykernel_147895/3542202393.py:15: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
/tmp/ipykernel_147895/3542202393.py:15: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
/tmp/ipykernel_147895/3542202393.py:15: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an un

In [57]:
COLOR_MAP

{'pz': '#D95319',
 'soz': '#A2142F',
 'nz': '#0072BD',
 'niz': '#0072BD',
 'pz_soz': '#D95319',
 'soz_soz': '#A2142F',
 'nz_soz': '#0072BD',
 'PZ': '#D95319',
 'SOZ': '#A2142F',
 'NZ': '#0072BD',
 'NIZ': '#0072BD',
 'PZ_SOZ': '#D95319',
 'SOZ_SOZ': '#A2142F',
 'NZ_SOZ': '#0072BD'}